<img style="float: left;" alt="Drawing" src="./figures/3Di_beeldmerk_RGB.png" width="100"/>

## From starting a 3Di-simulation to downloading and analysing the results in a jupyter notebook

Welcome! In this notebook we will show you how to start a <a href="https://3diwatermanagement.com/">3Di</a>-simulation in a jupyter notebook by using the API-v3. In addition, we will show you how to download, visualize and analyse the results of the 3Di-simulation.

The following steps will be taken according to an example of an 3Di model:
- **step 1:** Creating a 3Di-simulation by using the threedi-api
- **step 2:** Adding events to this 3Di-simulation by using the threedi-api
- **step 3:** Running the 3Di-simulation by using the threedi-api
- **step 4:** Downloading the results of the 3Di-simulation
- **step 5:** Analysing the results of the simulation

**Step 1: Starting a 3Di-simulation by using the threedi-api**

Importing all required packages:

In [2]:
from datetime import datetime
from getpass import getpass
import pandas as pd
import json
from threedi_api_client.threedi_api_client import ThreediApiClient
from threedi_api_client.api import ThreediApi
from threedi_api_client.versions import V3Api
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests
from pathlib import Path
import matplotlib.pylab as pylab
import numpy as np

Set some figures plot parameters:

In [3]:
plt.style.use('bmh')

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 15),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params) 

Provide your credentials to connect to the threedi-api. Logging in is done with the 3di api key, which can be generated and revoked via: https://management.3di.live/personal_api_keys. These need to be put into an .env file (example available in this map). Afterwards, you can log in with the following code: 

In [4]:
env_file = r"C:\Users\ththelen\OneDrive - North Carolina State University\CarolinaBeach\3Di\Scripts\authentication_TT.env"
THREEDI_API: V3Api = ThreediApi(env_file)

Check the connection with your provided credentials:

In [5]:
try:
    user = THREEDI_API.auth_profile_list()
except ApiException as e:
    print("Oops, something went wrong. Maybe you made a typo?")
else:
    print(f"Successfully logged in as {user.username}!")

Successfully logged in as ththelen!


1.3  In order to run a simulation you need a threedi-model. Let's see which threedi-models are available:

In [6]:
models = THREEDI_API.threedimodels_list(name__icontains='CB')  # limit to the first 10 results
for model in models.results:
    print(f"{model.name}")

CB_master #56
CB_master #54
CB_master #53
CB_NoDocks_V4 #4
CB_NoDocks_V4 #3
CB_NoDocks_V4 #2
Thelen_CB_3 #3
Thelen_CB_3 #2
Thelen_CB_3 #1
Thelen_CB_2 #1


In this notebook we will use one revision of the  3Di-model "BWN Schermer", and we can look up this model by using the following query:

In [7]:
models = THREEDI_API.threedimodels_list(name__icontains='CB_master #56')
my_model =models.results[0]
my_model

{'breach_count': '1',
 'description': '',
 'disabled': False,
 'epsg': 3358,
 'extent_one_d': {'coordinates': [[-77.89298758432503, 34.03613550525803],
                                  [-77.88447427775391, 34.05332936686093]],
                  'type': 'LineString'},
 'extent_two_d': {'coordinates': [[-77.91153736750825, 34.016244390386255],
                                  [-77.85633103791591, 34.10032463614724]],
                  'type': 'LineString'},
 'extent_zero_d': None,
 'id': 58509,
 'inp_success': True,
 'inpy_version': '-',
 'is_valid': True,
 'lines_count': 97460,
 'model_ini': ' ',
 'name': 'CB_master #56',
 'nodes_count': 48870,
 'repository_slug': 'cb_master',
 'revision': 'https://api.3di.live/v3/schematisations/6367/revisions/50244/',
 'revision_commit_date': '2023-07-17T18:44:51.142398Z',
 'revision_hash': '',
 'revision_id': 50244,
 'revision_number': '56',
 'schematisation_id': 6367,
 'schematisation_name': 'CB_master',
 'slug': 'cb_master-56',
 'storage_space': 

Now that we have a model we are almost ready to create the simulation. However, first we'll need to get an organisation under which's name we will run the simulation.

Let's see which organisations are available within my user account:

In [8]:
#organisations = THREEDI_API.organisations_list(name__icontains='N&S Demo')
organisations = THREEDI_API.organisations_list()

for organisation in organisations.results:
    print(f"{organisation.name}: {organisation.unique_id}")

organisation_uuid = organisation.unique_id

Academic License: 086a4dfe6fbe43698e4a320c5007c053


In [9]:
THREEDI_API.simulations_list(name__icontains='CB_master #56')

{'count': 2,
 'next': None,
 'previous': None,
 'results': [{'cloned_from': 'https://api.3di.live/v3/simulations/138845/',
              'compute_cluster': 'k8s',
              'created': '18 days ago',
              'duration': 3153600000,
              'duration_humanized': '36500 days, 0 hours, 0 minutes , 0 '
                                    'seconds',
              'end_datetime': datetime.datetime(2123, 6, 27, 2, 50, 53, tzinfo=tzutc()),
              'id': 139270,
              'name': 'CB_master #56_2023-07-21T105053',
              'organisation': '086a4dfe6fbe43698e4a320c5007c053',
              'organisation_name': 'Academic License',
              'slug': 'cb_master-56_2023-07-21t105053-aab9a36e-7683-4558-b47e-8977538ce1b0',
              'start_datetime': datetime.datetime(2023, 7, 21, 2, 50, 53, tzinfo=tzutc()),
              'tags': [],
              'threedicore_version': '3.2.49-3.0.0',
              'threedimodel': 'https://api.3di.live/v3/threedimodels/58509/',
  

1.5 Let's create the simulation of the chosen model now, with this organisation uuid. Note that it will not run yet.

In [10]:
my_simulation_template = THREEDI_API.simulation_templates_list(simulation__threedimodel__id=my_model.id).results[0]
my_simulation = THREEDI_API.simulations_from_template(
        data={
            "template": my_simulation_template.id,
            "name": "sim_cb_master", #3dinotebook_schermer",
            #"tags": #["demo_notebook_1"],
            "organisation": organisation_uuid,
            "start_datetime": datetime.now(),
            "duration": 3600 # in seconds, so we simulate for 1 hour
        }
    )

You can check the status of the simulation with the following api call:

In [11]:
#check the status of the simulation with:
status = THREEDI_API.simulations_status_list(my_simulation.id)
print(status)

{'created': datetime.datetime(2023, 8, 8, 12, 39, 39, 904153, tzinfo=tzutc()),
 'exit_code': None,
 'id': 649180,
 'name': 'created',
 'paused': None,
 'time': 0.0}


We can see the simulation has not started yet. The options at the name of the status can be: "created", "started" and "finished".

**Step 2: Adding events to this 3Di-simulation by using the threedi-api**

In the previous step we created a simulation for the 3Di model of rockflow. Several events can be added to this 3Di-simulation:

* initial waterlevels
* rain
* breaches
* laterals


In [18]:
THREEDI_API.simulations_events_structure_control_table_create(simulation_pk=my_simulation.id, data = 
        {
            "offset": 0,
            "duration": 3000000,
            "measure_specification": {
                "name": "Oystershell",
                "locations": [
                    {
                        "weight": "1.00",
                        "content_type": "v2_connection_node",
                        "content_pk": 5
                    }
                ],
                "variable": "s1",
                "operator": "<"
            },
            "structure_id": 2,
            "structure_type": "v2_weir",
            "type": "set_discharge_coefficients",
            "values": [
                [
                    0.5,
                    0.0010,
					0.6
                ],
                [
                    10,
                    0.0006,
					0.6
                ]
            ]
        }
        )

{'duration': 3000000,
 'grid_id': None,
 'id': 12359,
 'measure_specification': {'id': 12354,
                           'locations': [{'content_pk': 5,
                                          'content_type': 'v2_connection_node',
                                          'grid_id': None,
                                          'id': 12354,
                                          'state': 'processing',
                                          'state_detail': None,
                                          'weight': '1.00'}],
                           'name': 'Oystershell',
                           'operator': '<',
                           'variable': 's1'},
 'offset': 0,
 'state': 'processing',
 'state_detail': None,
 'structure_id': 2,
 'structure_type': 'v2_weir',
 'type': 'set_discharge_coefficients',
 'uid': '810b2ccb-7fb9-4d6c-8d10-f8e831391347',
 'url': 'https://api.3di.live/v3/simulations/142413/events/structure-control/table/12359/',
 'values': [[0.5, 0.001, 0.6], [

In [19]:
THREEDI_API.simulations_events_structure_control_table_create(simulation_pk=my_simulation.id, data =
		{
            "offset": 0,
            "duration": 3000000,
            "measure_specification": {
                "name": "Clamshell",
                "locations": [
                    {
                        "weight": "1.00",
                        "content_type": "v2_connection_node",
                        "content_pk": 4
                    }
                ],
                "variable": "s1",
                "operator": "<"
            },
            "structure_id": 3,
            "structure_type": "v2_weir",
            "type": "set_discharge_coefficients",
            "values": [
                				[
                    0.5,
                    0.0005,
					1
                ],
                [
                    10,
                    0.0002,
					1
                ],
            ]
        })                                

{'duration': 3000000,
 'grid_id': None,
 'id': 12360,
 'measure_specification': {'id': 12355,
                           'locations': [{'content_pk': 4,
                                          'content_type': 'v2_connection_node',
                                          'grid_id': None,
                                          'id': 12355,
                                          'state': 'processing',
                                          'state_detail': None,
                                          'weight': '1.00'}],
                           'name': 'Clamshell',
                           'operator': '<',
                           'variable': 's1'},
 'offset': 0,
 'state': 'processing',
 'state_detail': None,
 'structure_id': 3,
 'structure_type': 'v2_weir',
 'type': 'set_discharge_coefficients',
 'uid': 'd9bc572b-1b53-45c7-82ce-b3d811b1f14d',
 'url': 'https://api.3di.live/v3/simulations/142413/events/structure-control/table/12360/',
 'values': [[0.5, 0.0005, 1.0], [1

In [20]:
THREEDI_API.simulations_events_structure_control_table_create(simulation_pk=my_simulation.id, data =
		{
            "offset": 0,
            "duration": 3000000,
            "measure_specification": {
                "name": "CaptainsQuarters",
                "locations": [
                    {
                        "weight": "1.00",
                        "content_type": "v2_connection_node",
                        "content_pk": 32
                    }
                ],
                "variable": "s1",
                "operator": "<"
            },
            "structure_id": 4,
            "structure_type": "v2_weir",
            "type": "set_discharge_coefficients",
            "values": [
                				[
                    0.5,
                    0.0005,
					1
                ],
                [
                    10,
                    0.0002,
					1
                ],
            ]
        })                                

{'duration': 3000000,
 'grid_id': None,
 'id': 12361,
 'measure_specification': {'id': 12356,
                           'locations': [{'content_pk': 32,
                                          'content_type': 'v2_connection_node',
                                          'grid_id': None,
                                          'id': 12356,
                                          'state': 'processing',
                                          'state_detail': None,
                                          'weight': '1.00'}],
                           'name': 'CaptainsQuarters',
                           'operator': '<',
                           'variable': 's1'},
 'offset': 0,
 'state': 'processing',
 'state_detail': None,
 'structure_id': 4,
 'structure_type': 'v2_weir',
 'type': 'set_discharge_coefficients',
 'uid': '08d9a5c5-3604-4625-9cff-61809b2744f9',
 'url': 'https://api.3di.live/v3/simulations/142413/events/structure-control/table/12361/',
 'values': [[0.5, 0.0005, 

In [2]:
dataset = nc.Dataset(output_nc_file_path, 'r')

# Print the list of variables and dimensions
print("Variables:")
for var_name in dataset.variables:
    print(f"- {var_name}")

print("\nDimensions:")
for dim_name, dim in dataset.dimensions.items():
    print(f"- {dim_name}: {len(dim)}")

timestep_var = dataset.variables['timestep']
print("Timesteps:")
print(timestep_var[:10])

timestep_var = dataset.variables['value']
print("Values:")
print(timestep_var[:10])

Variables:
- timestep
- value

Dimensions:
- data_points: 3157
Timesteps:
[ 5 10 15 20 25 30 35 40 45 50]
Values:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [23]:
THREEDI_API.simulations_events_rain_timeseries_netcdf_create(simulation_pk=my_simulation.id, data =
    {"filename": output_nc_file_path})

{'filename': 'CUsersththelenDownloadsoutput.nc',
 'put_url': 'https://files.3di.live/3di/742912/CUsersththelenDownloadsoutput.nc?AWSAccessKeyId=3di&Signature=Sw8ZyJOI1OTXPEWCBjYu%2FWxCF6A%3D&Expires=1691590058',
 'status': None}

In [26]:
THREEDI_API.simulations_events_boundaryconditions_file_list(simulation_pk=my_simulation.id)

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'file': {'bucket': '3di',
                       'etag': None,
                       'expiry_date': datetime.date(2023, 8, 15),
                       'filename': 'CUsersththelenDownloadsoutput.json',
                       'id': 743108,
                       'meta': None,
                       'prefix': None,
                       'related_object': 'https://api.3di.live/v3/simulations/142446/events/boundaryconditions/file/57405/',
                       'size': None,
                       'state': 'created',
                       'state_description': None,
                       'storage_name': 'MINIO_DEV',
                       'type': 'bulk_boundaryconditions',
                       'url': 'https://api.3di.live/v3/files/743108/'},
              'id': 57405,
              'simulation': 'https://api.3di.live/v3/simulations/142446/',
              'state': 'processing',
              'state_detail': None,
            

In [29]:
# manually enter results[id] integert in for this API call ID number
THREEDI_API.simulations_events_boundaryconditions_file_delete(id = 57405, simulation_pk=my_simulation.id)
THREEDI_API.simulations_events_boundaryconditions_file_create(simulation_pk=my_simulation.id, data = 
{'filename' : json_file_path})

In [16]:
THREEDI_API.simulations_initial2d_water_level_constant_list(simulation_pk=my_simulation.id)

{'count': 0, 'next': None, 'previous': None, 'results': []}

In [18]:
THREEDI_API.simulations_initial2d_water_level_constant_delete(id = 34089, simulation_pk=my_simulation.id)
THREEDI_API.simulations_initial2d_water_level_constant_create(simulation_pk=my_simulation.id, data =
{"value": 1})

{'id': 34090,
 'simulation': 'https://api.3di.live/v3/simulations/142607/',
 'uid': '4354cedb-eaef-4b57-85dd-86545bb9e72c',
 'url': 'https://api.3di.live/v3/simulations/142607/initial/2d_water_level/constant/34090/',
 'value': 1.0}

2.3 We can get an overview of the added events to our 3Di-simulation by the following api-call:

In [20]:
events = THREEDI_API.simulations_events(my_simulation.id)
#print(events.timeseriesrain)
print(events)

{'breach': [],
 'fileboundaryconditions': {'file': {'bucket': '3di',
                                     'etag': 'f58cc182838b2937e2d44b668425902c',
                                     'expiry_date': datetime.date(2123, 6, 24),
                                     'filename': 'boundaries_faff6195.json',
                                     'id': 723060,
                                     'meta': {'event_count': 3,
                                              'timeframes': [0, 5999940]},
                                     'prefix': None,
                                     'related_object': 'https://api.3di.live/v3/simulations/138845/events/boundaryconditions/file/55088/',
                                     'size': 252,
                                     'state': 'processed',
                                     'state_description': 'File has been '
                                                          'uploaded '
                                                         

So, we can indeed see here that we have only added the constant rain event to our 3di-simulation.

**Step 3: Running the 3Di-simulation by using the threedi-api**

We will now start our simulation with the constant rain event:

In [18]:
THREEDI_API.simulations_actions_create(my_simulation.id, data={"name": "start"})

{'compute_cluster': None,
 'duration': None,
 'max_rate': None,
 'name': 'start',
 'timeout': 300}

We can check the status of the 3Di-simulation with:

In [19]:
#check the status of the simulation with:
status = THREEDI_API.simulations_status_list(my_simulation.id)
print(status)

{'created': datetime.datetime(2023, 1, 23, 16, 1, 47, 653607, tzinfo=tzutc()),
 'exit_code': None,
 'id': 519176,
 'name': 'starting',
 'paused': None,
 'time': 0.0}


In the end we must see that our simulation has finished:

In [20]:
#check the status of the simulation with:
status = THREEDI_API.simulations_status_list(my_simulation.id)
print(status)

{'created': datetime.datetime(2023, 1, 23, 16, 1, 47, 653607, tzinfo=tzutc()),
 'exit_code': None,
 'id': 519176,
 'name': 'starting',
 'paused': None,
 'time': 0.0}



**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
The end

In this tutorial we showed you:
- how to start a simulation of a 3Di-model, 
- how to add a simple rain event to your simulation, 
- how to download the results of the model simulation,
- and in the end how to visualise your model and how to use threedigrid to do any analysis on the results.

And all possible within a Jupyter notebook by making use of the API-v3. Ofcourse there are a lot more options for the analysis possible, by using threedigrid. Or for the events you can add to your simulation. Do you want to learn more or are you interested in some more possible analysis in a jupyter notebook? 

**Please contact**:

Olof Baltus | olof.baltus@nelen-schuurmans.nl

Jonas van Schrojenstein | jonas.vanschrojenstein@nelen-schuurmans.nl


**Utilites by TT**

In [ ]:
# Function to write .json boundary condition files (for API) from .csv files formatted for modeler interface upload

import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    data_list = []

    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip header row

        for row in csv_reader:
            id_value = int(row[0])
            timeseries = row[1].replace('"', '').split('\n')

            values = [list(map(float, point.split(','))) for point in timeseries]

            data_list.append({
                "id": id_value,
                "type": "2D",
                "interpolate": False,
                "values": values
            })

    with open(json_file_path, 'w') as json_file:
        json.dump(data_list, json_file, indent=4)

# Example usage:
csv_file_path = r"C:\Users\ththelen\OneDrive - North Carolina State University\CarolinaBeach\3Di\bcFiles\august2022\offset0_13\bc3di_tidesAndAtm_aug2022.csv"
json_file_path = r"C:\Users\ththelen\OneDrive - North Carolina State University\CarolinaBeach\3Di\bcFiles\august2022\offset0_13\bc3di_tidesAndAtm_aug2022.json"
csv_to_json(csv_file_path, json_file_path)

In [ ]:
# Function to write netCDF boundary condition files (for API) from .csv files formatted for modeler interface upload

import netCDF4 as nc

def csv_to_netcdf(csv_file_path, output_nc_file_path):
    # Read data from CSV file
    timesteps = []
    values = []
    with open(csv_file_path, 'r') as file:
        for line in file:
            timestep, value = line.strip().split(',')
            timesteps.append(int(timestep))
            values.append(float(value))
    
    # Create a new NetCDF file
    with nc.Dataset(output_nc_file_path, 'w', format='NETCDF4') as dataset:
        dataset.createDimension('data_points', len(timesteps))
        
        timestep_var = dataset.createVariable('timestep', 'i4', ('data_points',))
        value_var = dataset.createVariable('value', 'f8', ('data_points',))
        
        timestep_var[:] = timesteps
        value_var[:] = values

# Example usage:
csv_file_path = r"C:\Users\ththelen\OneDrive - North Carolina State University\CarolinaBeach\3Di\bcFiles\august2022\cbRain_20220813_20220824_5min.csv"
output_nc_file_path = r'C:\Users\ththelen\Downloads\output.nc'
csv_to_netcdf(csv_file_path, output_nc_file_path)

# Review results
dataset = nc.Dataset(output_nc_file_path, 'r')

# Print the list of variables and dimensions
print("Variables:")
for var_name in dataset.variables:
    print(f"- {var_name}")

print("\nDimensions:")
for dim_name, dim in dataset.dimensions.items():
    print(f"- {dim_name}: {len(dim)}")

timestep_var = dataset.variables['timestep']
print("Timesteps:")
print(timestep_var[:10])

timestep_var = dataset.variables['value']
print("Values:")
print(timestep_var[:10])